Создания классификатора на сайтах MailTop100

In [0]:
import pandas as pd
import numpy as np
from collections import Counter

In [0]:
from multiprocessing.dummy import Pool
import concurrent.futures
import multiprocessing
import tqdm

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [0]:
import gensim
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier

Сборка датасетов

In [0]:
df = pd.read_csv('/content/drive/My Drive/mailClassifier/mail_lemmatized_FULL_v2.csv', encoding = 'utf-8-sig')
df = df.drop('Unnamed: 0', 1)
df = df.fillna('')
df = df[df['content']!='']
df = df[df['category']!='WAP сайты']

df['titledesc'] = df[['title','description']].apply(lambda row: ' '.join(row),axis=1)

Сборка датасета категорий (df_categs)

In [0]:
df_categs = df[['category','titledesc','content']]
df_categs = df_categs[df_categs['titledesc']!='']

In [0]:
dict_categs = {}
dict_categsINV = {}
categs = list(set(df_categs['category']))

for i in range(17):
  dict_categs[i] = categs[i]

for i in range(17):
  dict_categsINV[categs[i]] = i

In [0]:
dict_categs

In [0]:
df_categs = df_categs.replace({'category': dict_categsINV})

In [0]:
df_categs

In [0]:
df_categs.groupby(['category']).size().reset_index(name='counts').sort_values(by=['counts'])

Сборка датасета подкатегорий (df_subcategs)

In [0]:
df_subcategs = df[['subcategory','titledesc','content']]
df_subcategs = df_subcategs[df_subcategs['titledesc']!='']

subcateg_counts = df_subcategs.groupby(['subcategory']).size().reset_index(name='counts').sort_values(by=['counts'])
indeed_subcategs = set(subcateg_counts[subcateg_counts['counts']>=500]['subcategory'])

df_subcategs = df_subcategs[df_subcategs['subcategory'].isin(indeed_subcategs)]
df_subcategs = df_subcategs.rename(columns={'subcategory':'category'})

In [0]:
df_subcategs = df_subcategs[df_subcategs['titledesc']!='']
df_subcategs = df_subcategs[df_subcategs['content']!='']
df_subcategs = df_subcategs.reset_index(drop=True)

In [0]:
dict_subcategs = {}
dict_subcategsINV = {}
subcategs = list(set(df_subcategs['category']))

for i in range(len(subcategs)):
  dict_subcategs[i] = subcategs[i]

for i in range(len(subcategs)):
  dict_subcategsINV[subcategs[i]] = i

In [0]:
df_subcategs = df_subcategs.replace({'category': dict_subcategsINV})

In [10]:
dict_subcategs

{0: 'Информация',
 1: 'Домашний доктор',
 2: 'Научные издания и публикации',
 3: 'Фотоискусство',
 4: 'Семейный досуг',
 5: 'Киноискусство',
 6: 'Банки и пункты обмена валют',
 7: 'Журналы',
 8: 'Софт',
 9: 'Купля-продажа',
 10: 'Путешествия',
 11: 'Школы',
 12: 'Науки общественные',
 13: 'Общение',
 14: 'Единоборства',
 15: 'Химия и нефтехимия',
 16: 'Города от А до Я',
 17: 'Туристические агентства, фирмы',
 18: 'Домашние животные',
 19: 'Железо',
 20: 'Медицинская промышленность',
 21: 'Детская комната',
 22: 'Страны',
 23: 'Автоматизация в промышленности',
 24: 'Музыка',
 25: 'Гостиничные услуги',
 26: 'Связь, операторы связи',
 27: 'Усадьба',
 28: 'ТВ',
 29: 'Психология',
 30: 'Регионы',
 31: 'Радиоэлектронная промышленность и приборостроение',
 32: 'Общение, встречи, контакты',
 33: 'Кино/видео/фото',
 34: 'Торговля',
 35: 'Литература',
 36: 'Операции с недвижимостью',
 37: 'Веб-дизайн',
 38: 'Активный отдых',
 39: 'Религия',
 40: 'Финансовые операции, денежное обращение',
 41: '

In [11]:
df_subcategs

,category,titledesc,content
0,26,сетевой устройство купить сетевой оборудование...,включить браузер каталог товар блог статья обм...
1,18,персональный чудокотик,порода чудо котик папа чудо котик мама купить ...
2,45,проект ардуин урок русский начинающий проект у...,распиновка урок статья курс библиотека справоч...
3,60,репетитор скайп изучение иностранный язык скай...,репетитор скайп изучать иностранный язык изуче...
4,78,дмш гволжский,меню новость школа портфолио отдел народный от...
...,...,...,...
191527,30,администрация тамбовский район,район общий сведение символика учреждение райо...
191528,11,центр иностранный язык высокий стандарт обучен...,компания новость отзыв преподаватель вакансия ...
191529,83,отделка квартира ремонт квартира евроремонт ре...,цена контакт фото гостевой каталог рейтинг чес...
191530,118,красивый стать легко,стать красивый успешный счастливый похудеть со...


**Вспомогательные функции**

Две функции нахождения самых популярных слов:<br>
1)Первая функция достает указанное колличество самых популярных слов для каждой категории и записывает в общий список, далее возвращает его. Делает отдельно для столбцов titledesc и content <br>
2)Вторая функция достает указанное колличество слов среди всех категорий, также отдельно для titledesc и content

In [0]:
def FindMCWordsByCategory(ourdf, numberTitle, numberContent):
  dfgroups_content = ourdf[['category','content']].groupby('category')['content'].apply(lambda row: ' '.join(row))
  dfgroups_title = ourdf[['category','titledesc']].groupby('category')['titledesc'].apply(lambda row: ' '.join(row))
  mostCW_content = []
  mostCW_title = []
  for i in range(len(dfgroups_content)):
    counter1 = Counter(dfgroups_content[i].split())
    most_occur = counter1.most_common(numberContent)
    mostCW = [ocur[0] for ocur in most_occur]
    mostCW_content.extend(mostCW)

    counter2 = Counter(dfgroups_title[i].split())
    most_occur = counter2.most_common(numberTitle)
    mostCW = [ocur[0] for ocur in most_occur]
    mostCW_title.extend(mostCW)

  return mostCW_title, mostCW_content

def FindMCWords(ourdf, numberTitle, numberContent):
  
  corpusstr = ' '.join(ourdf['content'])
  counter1 = Counter(corpusstr.split())
  most_occur = counter1.most_common(numberContent)
  mostCW = [ocur[0] for ocur in most_occur]
  
  mostCW_content = mostCW

  corpusstr = ' '.join(ourdf['titledesc'])
  counter1 = Counter(corpusstr.split())
  most_occur = counter1.most_common(numberTitle)
  mostCW = [ocur[0] for ocur in most_occur]

  mostCW_title = mostCW

  return mostCW_title, mostCW_content

Создаем две функции для удаления слов

In [0]:
def SelectWords(text):
  tokens = text.split()
  tokens = [token for token in tokens if token in WordsToSelect]
  
  return ' '.join(tokens)

def DeleteWords(text):
  tokens = text.split()
  tokens = [token for token in tokens if token not in WordsToDelete]
  
  return ' '.join(tokens)

**Отбрасывание самых редких и самых частых слов**

In [0]:
dfToClassifier = df_subcategs.copy()

Убираем все совпадающие слова в каждой строке

In [0]:
dfToClassifier['titledesc'] = dfToClassifier['titledesc'].apply(lambda row: ' '.join(list(set(row.split()))))
dfToClassifier['content'] = dfToClassifier['content'].apply(lambda row: ' '.join(list(set(row.split()))))

Используя созданные ранее функции создаем списки из слов для titledesc и content<br>
В список для titledesc засовываем 100 самых частых слов для каждой категории из столбца titledesc <br>
А для content - 750 самых частых из столбца content<br>
После этого оставляем только те слова, которые есть в этих двух списках, используя мультипроцессинг

In [16]:
CWByCategory_title, CWByCategory_content = FindMCWordsByCategory(dfToClassifier,100,750)

CWByCategory_content = set(CWByCategory_content)
CWByCategory_title = set(CWByCategory_title)

content = dfToClassifier['content'].to_list()
titles = dfToClassifier['titledesc'].to_list()

content_length = len(content)

WordsToSelect = CWByCategory_content
with concurrent.futures.ProcessPoolExecutor() as executor:
  r = list(tqdm.tqdm(executor.map(SelectWords, content), total=content_length))

dfToClassifier['content'] = r

WordsToSelect = CWByCategory_title
with concurrent.futures.ProcessPoolExecutor() as executor:
  r = list(tqdm.tqdm(executor.map(SelectWords, titles), total=content_length))

dfToClassifier['titledesc'] = r

100%|██████████| 191532/191532 [00:14<00:00, 13482.54it/s]


Убираем все пустые строки

In [0]:
dfToClassifier = dfToClassifier[dfToClassifier['titledesc']!='']
dfToClassifier = dfToClassifier[dfToClassifier['content']!='']
dfToClassifier = dfToClassifier.reset_index(drop=True)

Убираем самые частые слова среди всех категорий (100 для titledesc, 250 для content)

In [18]:
CWtitle, CWcontent = FindMCWords(dfToClassifier,100,250)
CWcontent = set(CWcontent)
CWtitle = set(CWtitle)

content = dfToClassifier['content'].to_list()
titles = dfToClassifier['titledesc'].to_list()
content_length = len(content)

WordsToDelete = CWcontent
with concurrent.futures.ProcessPoolExecutor() as executor:
  r = list(tqdm.tqdm(executor.map(DeleteWords, content), total=content_length))
dfToClassifier['content'] = r

WordsToDelete = CWtitle
with concurrent.futures.ProcessPoolExecutor() as executor:
  r = list(tqdm.tqdm(executor.map(DeleteWords, titles), total=content_length))
dfToClassifier['titledesc'] = r

100%|██████████| 170184/170184 [00:11<00:00, 14940.40it/s]


Убираем пустые строки

In [19]:
dfToClassifier = dfToClassifier[dfToClassifier['titledesc']!='']
dfToClassifier = dfToClassifier[dfToClassifier['content']!='']
dfToClassifier = dfToClassifier.reset_index(drop=True)
print('DF Length: ',len(dfToClassifier))

DF Length:  157253


In [0]:
dfToClassifier = dfToClassifier[dfToClassifier['content'].str.split().str.len() > 10]
dfToClassifier = dfToClassifier[dfToClassifier['titledesc']!='']

**Представление текста через TF-IDF**

Оставляем только те строки, колличество слов content которых больше 10<br>
После чего делим выборку на трейн тест, при помощи метода

In [21]:
y = dfToClassifier['category'].tolist()

df_train, df_test, y_train, y_test = train_test_split(dfToClassifier, y, test_size=0.25, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

print('DF to train Length:',len(dfToClassifier))
print('y_train number:',len(set(y_train)))

DF to train Length: 151426
y_train number: 121


Создаем из df_train две матрицы<br>
Первая - TF-IDF матрица создается из сгруппированного по категориям df_train, используя метод fit.transform объекта TfidfVectorizer(), эта матрица имеет размерность КОЛ-ВО КАТЕГОРИЙ(121) x КОЛ-ВО СЛОВ В КОРПУСЕ<br>
Вторая - BOW матрица создается просто из df_train, через применение метода fit.transform объекта CountVectorizer(), эта матрица имеет размерность КОЛ-ВО СТРОК(ПРЕДЛОЖЕНИЙ) * КОЛ-ВО СЛОВ В КОРПУСЕ<br>
Теперь перемножаем эти две матрицы, для этого стоит транспонировать BOW матрицу<br> После успешного перемножения и повторного транспонирования перемноженная матрица будет иметь размерность КОЛ-ВО СТРОК(ПРЕДЛОЖЕНИЙ) x КОЛ-ВО КАТЕГОРИЙ(121)<br><br>
Проделываем тоже самое с df_test, только вместо fit.transform применяем просто transform, создать матрицу на той информации, что дал df_train<br><br>
Проделываем эти методы отдельно на titledesc и content, полученные матрицы конкатенируем<br>В результате получилось 2 матрицы train и test обе имеют размерность КОЛ-ВО СТРОК (трейн или тест) X 2xКОЛ-ВО КАТЕГОРИЙ(242)

In [0]:
tfidf_vectorizer1 = TfidfVectorizer()
tfidf_vectorizer2 = TfidfVectorizer()

countVectorizer1 = CountVectorizer()
countVectorizer2 = CountVectorizer()

count_matrix1 = countVectorizer1.fit_transform(df_train['titledesc'])
count_matrix2 = countVectorizer2.fit_transform(df_train['content'])

df_trainGroups1 = df_train[['category','titledesc']].groupby('category')['titledesc'].apply(lambda row: ' '.join(row)).reset_index(drop=True)
df_trainGroups2 = df_train[['category','content']].groupby('category')['content'].apply(lambda row: ' '.join(row)).reset_index(drop=True) 

tfidf_matrix1 = tfidf_vectorizer1.fit_transform(df_trainGroups1)
tfidf_matrix2 = tfidf_vectorizer2.fit_transform(df_trainGroups2)

train_matrix1 = tfidf_matrix1 * count_matrix1.T
train_matrix2 = tfidf_matrix2 * count_matrix2.T

train_matrix = np.concatenate((train_matrix1.T.toarray(),train_matrix2.T.toarray()),axis=1)

count_matrix_test1 = countVectorizer1.transform(df_test['titledesc'])
count_matrix_test2 = countVectorizer2.transform(df_test['content'])

test_matrix1 = tfidf_matrix1 * count_matrix_test1.T
test_matrix2 = tfidf_matrix2 * count_matrix_test2.T

test_matrix = np.concatenate((test_matrix1.T.toarray(),test_matrix2.T.toarray()),axis=1)

In [23]:
X_train = train_matrix
X_test = test_matrix

print(X_train.shape)
print(X_test.shape)

(113569, 242)
(37857, 242)


In [24]:
print('### Logreg: ###')
logreg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
print('ACCURACY :', accuracy_score(y_pred, y_test))
print('F1 :', f1_score(y_pred, y_test, average='macro'))

### Logreg: ###
ACCURACY : 0.4846531776639019
F1 : 0.4378097551573957


**Представление слов через fasttext word embeddings**

Подключаем вектора предобученной модели fasttext

In [0]:
model = gensim.models.KeyedVectors.load('/content/drive/My Drive/mailClassifier/fasttext_v2/model.model')

In [0]:
fastext_vocab = set(model.wv.vocab)

Вариант перевода предложений в векторное представление путем нахождения среднего вектора между векторами всех слов текста (предложения) 

In [0]:
def text_to_vector(text):
  doc = [word for word in text.split() if word in fastext_vocab]
  
  if len(doc)!=0:
    return np.mean(model[doc],axis=0)
  else:
    return np.zeros((300,))

In [0]:
a = dfToClassifier['titledesc'].apply(lambda row: text_to_vector(row))
b = dfToClassifier['content'].apply(lambda row: text_to_vector(row))

In [29]:
X = np.concatenate((a.tolist(),b.tolist()),axis=1)
y = dfToClassifier['category'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print('y_train number:',len(set(y_train)))
print(len(X_train), len(X_test))

y_train number: 121
113569 37857


In [30]:
print('### Logreg: ###')
logreg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
print('ACCURACY :', accuracy_score(y_pred, y_test))
print('F1 :', f1_score(y_pred, y_test, average='macro'))

### Logreg: ###
/usr/local/lib/python3.6/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
ACCURACY : 0.4311032072116269
F1 : 0.3818336418720104
